# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118732e+02     2.120767e+00
 * time: 0.08827900886535645
     1     1.016047e+01     1.086308e+00
 * time: 0.9586811065673828
     2    -1.199763e+01     1.265166e+00
 * time: 1.097465991973877
     3    -3.413416e+01     1.025769e+00
 * time: 1.2654049396514893
     4    -4.766036e+01     6.577692e-01
 * time: 1.4148359298706055
     5    -5.710637e+01     2.053705e-01
 * time: 1.5604910850524902
     6    -5.988830e+01     1.803777e-01
 * time: 1.6670520305633545
     7    -6.095313e+01     5.293264e-02
 * time: 1.780668020248413
     8    -6.138593e+01     6.561034e-02
 * time: 1.8952720165252686
     9    -6.164354e+01     5.034667e-02
 * time: 2.0031919479370117
    10    -6.185275e+01     2.900325e-02
 * time: 2.1173629760742188
    11    -6.199989e+01     1.935438e-02
 * time: 2.2253670692443848
    12    -6.207010e+01     1.551784e-02
 * time: 2.3411519527435303
    13    -6.211762e+01     1.480023e-02
 * time: 2.45492196083

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557685
    AtomicNonlocal      14.8522644
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336823

    total               -62.261666458887
